In [1]:
from transformers import AutoTokenizer, MT5ForConditionalGeneration
import torch
import numpy as np
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm 
import matplotlib.pyplot as plt
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import defaultdict
import json

/home/gpu/anaconda3/envs/news_classifier/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-16 15:42:45.273420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 15:42:45.883878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = load_dataset("Someman/news_nepali")

In [3]:
dataset['train']

Dataset({
    features: ['article', 'article_summary'],
    num_rows: 15580
})

In [4]:
dataset['train'][0]['article']

'काठमाडौं, चैत ११ : सत्तारुढ नेकपा (माओवादी केन्द्र)ले चैत ११ गते (सोमबार)भित्र मन्त्रिपरिषद्लाई पूर्णता दिने निर्णय गरेको छ ।शनिबार दिउँसो पेरिसडाँडामा बसेको पार्टी पदाधिकारी बैठकले उक्त निर्णय गरेको हो । माओवादी केन्द्रको केन्द्रीय कार्यालय। पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए । सचिव पौडेलले प्रदेश सरकारलाई सङ्घीय सरकारको नीति तथा कार्यक्रमसँग जोडेर गतिशिल बनाउनुपर्नेमा नेताहरुले जोड दिएको जानकारी दिँदै पार्टी नेताहरुले प्रधानमन्त्री पुष्पकमल दाहालको पनि ध्यानाकर्षण गराए । पार्टी र सरकारबीचको खाडल घटाउन पार्टी अध्यक्ष डा.\r'

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small",use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
encoded_input = tokenizer(dataset['train'][0]['article'])

In [7]:
encoded_input

{'input_ids': [259, 55781, 261, 6158, 155844, 259, 69708, 259, 267, 259, 81877, 30096, 11812, 259, 43041, 9654, 274, 1195, 17996, 34834, 259, 42657, 271, 1048, 6158, 155844, 259, 69708, 1462, 1484, 274, 31943, 1981, 15028, 271, 59155, 75063, 3081, 17768, 74153, 41650, 4157, 259, 17514, 1114, 5831, 757, 15649, 3225, 1693, 2661, 3596, 2237, 259, 378, 17862, 3081, 15028, 5831, 85136, 31943, 21799, 102577, 14438, 105220, 53740, 21681, 3596, 259, 24794, 259, 156795, 102042, 9255, 4738, 70573, 2139, 24164, 15649, 3225, 1693, 2661, 3596, 1248, 259, 378, 5252, 17996, 34834, 259, 42657, 1131, 259, 42657, 7163, 8949, 14322, 378, 259, 24794, 1048, 8079, 1981, 15028, 25894, 75063, 3081, 17768, 74153, 41650, 259, 83570, 21889, 8108, 757, 38006, 124662, 150226, 15649, 3225, 1693, 2661, 3596, 259, 36603, 33608, 56603, 61898, 2745, 3597, 20462, 135912, 10429, 14148, 259, 36043, 259, 378, 259, 36603, 33608, 2745, 3597, 20462, 135912, 18175, 7141, 4157, 259, 94303, 10915, 7163, 7141, 1131, 259, 56631, 2

In [8]:
tokenizer.decode(encoded_input["input_ids"])

'काठमाडौं, चैत ११ : सत्तारुढ नेकपा (माओवादी केन्द्र)ले चैत ११ गते (सोमबार)भित्र मन्त्रिपरिषद्लाई पूर्णता दिने निर्णय गरेको छ ।शनिबार दिउँसो पेरिसडाँडामा बसेको पार्टी पदाधिकारी बैठकले उक्त निर्णय गरेको हो । माओवादी केन्द्रको केन्द्रीय कार्यालय। पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए । सचिव पौडेलले प्रदेश सरकारलाई सङ्घीय सरकारको नीति तथा कार्यक्रमसँग जोडेर गतिशिल बनाउनुपर्नेमा नेताहरुले जोड दिएको जानकारी दिँदै पार्टी नेताहरुले प्रधानमन्त्री पुष्पकमल दाहालको पनि ध्यानाकर्षण गराए । पार्टी र सरकारबीचको खाडल घटाउन पार्टी अध्यक्ष डा.</s>'

In [9]:
def summarizer(text,ml):
  mt5_prepared_text = "summarize: "+text
  tokenized_text = tokenizer.encode(mt5_prepared_text,return_tensors="pt").to(device)
  summary_ids = model.generate(tokenized_text,
                              num_beams=8,
                              no_repeat_ngram_size=3,
                              min_length=30,
                              max_length=ml,
                              length_penalty=0.1,
                              num_return_sequences=1,
                              early_stopping=True
                              )
  output = tokenizer.decode(summary_ids[0],skip_special_tokens=True)
  return output

In [10]:
## Finding the average summary length in our dataset
avg_summ_length = dataset['train']['article_summary']
avg_summ_length = [len(summary) for summary in avg_summ_length]
print(f'Average summary length: {np.mean(avg_summ_length)}')

Average summary length: 212.97150192554557


In [11]:
class SummarizationDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {"input_text": self.data[idx]["article"], "summary_text": self.data[idx]["article_summary"]}

## loading the dataset
train_dataset = SummarizationDataset(dataset['train'])
test_dataset = SummarizationDataset(dataset['test'])

# fine-tuning objective (CrossEntropyLoss for sequence-to-sequence tasks)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
# Defining optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Parameters
num_epochs = 30
patience = 3  # Stop training if validation loss does not improve for 'patience' epochs
best_val_loss = float('inf')  # Initialize best validation loss to infinity
early_stop_counter = 0  # Counter for early stopping

# Training and validation data loaders
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True)
val_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

# Initialize lists to store metrics
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

def calculate_accuracy(logits, labels):
    """Calculate token-level accuracy."""
    predictions = logits.argmax(dim=-1)  # Get token predictions
    correct = (predictions == labels) & (labels != tokenizer.pad_token_id)  # Exclude PAD tokens
    accuracy = correct.sum().item() / (labels != tokenizer.pad_token_id).sum().item()
    return accuracy


In [12]:

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    # Training loop
    model.train()
    total_train_loss = 0
    total_train_accuracy = 0

    train_bar = tqdm(train_dataloader, desc="Training", leave=False)
    for batch in train_bar:
        inputs = tokenizer(batch["input_text"], return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
        labels = tokenizer(batch["summary_text"], return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels["input_ids"])
        logits = outputs.logits

        # Flatten logits and labels
        logits_flat = logits.view(-1, logits.shape[-1])
        labels_flat = labels["input_ids"].view(-1)

        # Compute loss
        loss = loss_fn(logits_flat, labels_flat)
        total_train_loss += loss.item()

        # Calculate accuracy
        total_train_accuracy += calculate_accuracy(logits, labels["input_ids"])

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_bar.set_postfix({"Loss": loss.item()})

    # Compute average train loss and accuracy
    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)

    # Validation loop
    model.eval()
    total_val_loss = 0
    total_val_accuracy = 0

    val_bar = tqdm(val_dataloader, desc="Validating", leave=False)
    with torch.no_grad():
        for batch in val_bar:
            inputs = tokenizer(batch["input_text"], return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
            labels = tokenizer(batch["summary_text"], return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

            outputs = model(**inputs, labels=labels["input_ids"])
            logits = outputs.logits

            logits_flat = logits.view(-1, logits.shape[-1])
            labels_flat = labels["input_ids"].view(-1)

            # Compute validation loss
            loss = loss_fn(logits_flat, labels_flat)
            total_val_loss += loss.item()

            # Calculate accuracy
            total_val_accuracy += calculate_accuracy(logits, labels["input_ids"])

            val_bar.set_postfix({"Loss": loss.item()})

    # Compute average validation loss and accuracy
    avg_val_loss = total_val_loss / len(val_dataloader)
    avg_val_accuracy = total_val_accuracy / len(val_dataloader)

    print(f"Epoch {epoch + 1}/{num_epochs} Completed")
    print(f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    print(f"Train Accuracy: {avg_train_accuracy:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")

    # Append metrics
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(avg_train_accuracy)
    val_accuracies.append(avg_val_accuracy)

    # Early stopping logic
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        print("Validation loss improved, saving the model...")
        torch.save(model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        print(f"No improvement in validation loss for {early_stop_counter} epoch(s).")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Stopping training.")
        break

# Plotting loss and accuracy
# Save Loss Comparison Graph
plt.figure(figsize=(8, 6))
plt.plot(range(len(train_losses)), train_losses, label="Train Loss")
plt.plot(range(len(val_losses)), val_losses, label="Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Comparison")
plt.legend()
plt.grid(True)
plt.savefig("loss_comparison.png", dpi=300)  # Save as PNG
plt.savefig("loss_comparison.pdf")  # Save as PDF
plt.close()  # Close the figure to free memory

# Save Accuracy Comparison Graph
plt.figure(figsize=(8, 6))
plt.plot(range(len(train_accuracies)), train_accuracies, label="Train Accuracy")
plt.plot(range(len(val_accuracies)), val_accuracies, label="Val Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison")
plt.legend()
plt.grid(True)
plt.savefig("accuracy_comparison.png", dpi=300)  # Save as PNG
plt.savefig("accuracy_comparison.pdf")  # Save as PDF
plt.close()  # Close the figure to free memory


Epoch 1/30


Training:   0%|          | 0/15580 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/30 Completed
Train Loss: 3.9959, Val Loss: 1.1153
Train Accuracy: 0.4252, Val Accuracy: 0.8005
Validation loss improved, saving the model...

Epoch 2/30


Epoch 2/30 Completed
Train Loss: 1.4322, Val Loss: 0.9949
Train Accuracy: 0.7310, Val Accuracy: 0.8269
Validation loss improved, saving the model...

Epoch 3/30


...

Epoch 26/30 Completed
Train Loss: 0.6786, Val Loss: 0.6993
Train Accuracy: 0.8517, Val Accuracy: 0.8622
Validation loss improved, saving the model...

Epoch 27/30


Epoch 27/30 Completed
Train Loss: 0.6717, Val Loss: 0.7031
Train Accuracy: 0.8527, Val Accuracy: 0.8619
No improvement in validation loss for 1 epoch(s).

Epoch 28/30


Epoch 28/30 Completed
Train Loss: 0.6650, Val Loss: 0.6967
Train Accuracy: 0.8535, Val Accuracy: 0.8627
Validation loss improved, saving the model...

Epoch 29/30


Epoch 29/30 Completed
Train Loss: 0.6572, Val Loss: 0.6813
Train Accuracy: 0.8548, Val Accuracy: 0.8632
Validation loss improved, saving the model...

Epoch 30/30


Epoch 30/30 Completed
Train Loss: 0.6520, Val Loss: 0.6903
Train Accuracy: 0.8555, Val Accuracy: 0.8640
No improvement in validation loss for 1 epoch(s).


In [13]:

# Save the fine-tuned model
model.save_pretrained("fine_tuned_mt5_summarization_model")

# Save the tokenizer
tokenizer.save_pretrained("fine_tuned_mt5_summarization_model/tokenizer")

('fine_tuned_mt5_summarization_model/tokenizer/tokenizer_config.json',
 'fine_tuned_mt5_summarization_model/tokenizer/special_tokens_map.json',
 'fine_tuned_mt5_summarization_model/tokenizer/spiece.model',
 'fine_tuned_mt5_summarization_model/tokenizer/added_tokens.json')

In [ ]:
# Load the fine-tuned model and tokenizer
model = MT5ForConditionalGeneration.from_pretrained("fine_tuned_mt5_summarization_model").to(device)
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_mt5_summarization_model/tokenizer")

# Example input text
input_text = dataset['train'][0]['article']

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True).input_ids.to(device)

# Generate the summary
max_summary_length = 1000
summary_ids = model.generate(input_ids,max_length=max_summary_length,max_new_tokens=max_summary_length)

# Decode the generated summary
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/gpu/anaconda3/envs/news_classifier/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Both `max_new_tokens` (=300) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Summary: सत्तारुढ नेकपा (माओवादी केन्द्र)ले चैत ११ गते (सोमबार)भित्र मन्त्रिपरिषद्लाई पूर्णता दिने निर्णय गरेको छ । पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए ।


In [ ]:
# Initialize ROUGE scorer
rs = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Smoothing function for BLEU
smooth = SmoothingFunction().method1

# Store scores
rouge_scores = []
bleu_scores = []
scores = {}
# Iterate through validation data
for batch in tqdm(val_dataloader):
    # Get inputs and references from batch
    inputs = tokenizer(batch["input_text"], return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
    labels = tokenizer(batch["summary_text"], return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    
    # Generate predictions
    outputs = model.generate(inputs["input_ids"], max_length=1000, num_beams=5, early_stopping=True)
    
    # Decode predictions and references
    decoded_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    decoded_references = tokenizer.batch_decode(labels["input_ids"], skip_special_tokens=True)
    
    # Calculate ROUGE and BLEU for each pair
    for pred, ref in zip(decoded_predictions, decoded_references):
        # ROUGE score
        rouge_score = rs.score(pred, ref)
        rouge_scores.append(rouge_score)
        
        # BLEU score
        ref_tokens = ref.split()  # Tokenize reference
        pred_tokens = pred.split()  # Tokenize prediction
        bleu_score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth)
        bleu_scores.append(bleu_score)

# Average the scores
avg_rouge = {
    key: sum(score[key].fmeasure for score in rouge_scores) / len(rouge_scores)
    for key in ['rouge1', 'rouge2', 'rougeL']
}
avg_bleu = sum(bleu_scores) / len(bleu_scores)
scores["average_rogue_score"] = avg_rouge
scores["average_BLEU_score"]  = avg_bleu

with open("summary_metrics.json", "w") as jfile:
    json.dump(scores, jfile, indent=4)

# Print results
print("Average ROUGE Scores:", avg_rouge)
print("Average BLEU Score:", avg_bleu)

100%|██████████| 1732/1732 [31:28<00:00,  1.09s/it]

Average ROUGE Scores: {'rouge1': 0.004175831225484805, 'rouge2': 0.0018668206312548112, 'rougeL': 0.004175831225484805}
Average BLEU Score: 0.32817689445874376


In [16]:
generated_summary

'सत्तारुढ नेकपा (माओवादी केन्द्र)ले चैत ११ गते (सोमबार)भित्र मन्त्रिपरिषद्लाई पूर्णता दिने निर्णय गरेको छ । पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए ।'

In [17]:
dataset['train'][0]['article_summary']

'शनिबार दिउँसो पेरिसडाँडामा बसेको पार्टी पदाधिकारी बैठकले यस्तो निर्णय गरेको हो । पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए ।'

In [18]:
# def summary(text):

#    #input_text = dataset['train'][6]['article']

#   # Tokenize the input text
#   input_ids = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True).input_ids

#   # Generate the summary
#   max_summary_length = 150
#   summary_ids = model.generate(input_ids,max_length=max_summary_length,max_new_tokens=max_summary_length)


#   # Decode the generated summary
#   generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

#   # print("Generated Summary:", generated_summary)
#   return generated_summary


In [ ]:

# summary_demo = summary(dataset['train'][0]['article'])
# summary_demo

In [ ]:
def summarize_article(article):
    inputs = tokenizer(article, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
    summary = model.generate(inputs['input_ids'], max_length=1000)  # Using model's generate method for summarization
    decoded_summary = tokenizer.decode(summary[0], skip_special_tokens=True)
    return decoded_summary

# Example usage
summary_demo = summarize_article(dataset['train'][0]['article'])
print(summary_demo)

सत्तारुढ नेकपा (माओवादी केन्द्र)ले चैत ११ गते (सोमबार)भित्र मन्त्रिपरिषद्लाई पूर्णता दिने निर्णय गरेको छ । पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए ।


In [21]:
summary_demo


'सत्तारुढ नेकपा (माओवादी केन्द्र)ले चैत ११ गते (सोमबार)भित्र मन्त्रिपरिषद्लाई पूर्णता दिने निर्णय गरेको छ । पार्टीले सोमबारसम्म मन्त्रिपरिषद् विस्तार गर्ने औपचारिक निर्णय गरेको सचिव देवेन्द्र पौडेलले जानकारी दिए ।'

In [22]:
print(f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

Train Loss: 0.6520, Val Loss: 0.6903
